In [1]:
import os
import json
import requests
import pandas as pd

In [3]:
concept_list = ["Symmetric Encryption"]
field = "Cybersecurity"
level_list = ["Remember", "Understand", "Apply", "Analyze", "Evaluate", "Create"]
concept = concept_list[0]
level = level_list[0]

In [4]:
# step 1: prefetch knowledge component
knowledge_result = requests.post(
        url="http://18.206.229.27:8000/tree/create_knowledge_component",
        data=json.dumps({
            "concept": concept,
            "field": field,
            "level": level,
            "cache": "true"
        }))
knowledge_text = json.loads(knowledge_result.text)["data"]["knowledge"]

In [7]:
# step 2: prefetch knowledge graph
graph_result = requests.post(
        url="http://18.206.229.27:8000/tree/create_knowledge_graph",
        data=json.dumps({
            "concepts": [concept],
            "field": field,
            "knowledge": knowledge_text,
            "cache": "true"
        }))
graph_text = json.loads(graph_result.text)["data"]["graph"]

[{'source': 'Symmetric encryption', 'target': 'Encryption', 'relation': 'Type of'}, {'source': 'Symmetric encryption', 'target': 'Key', 'relation': 'Used for'}, {'source': 'Symmetric encryption', 'target': 'Data', 'relation': 'Encrypted and decrypted'}, {'source': 'Symmetric encryption', 'target': 'DES', 'relation': 'Type of algorithm'}, {'source': 'Symmetric encryption', 'target': 'AES', 'relation': 'Type of algorithm'}, {'source': 'Symmetric encryption', 'target': 'Blowfish', 'relation': 'Type of algorithm'}, {'source': 'Symmetric encryption', 'target': 'RC4', 'relation': 'Type of algorithm'}, {'source': 'Symmetric encryption', 'target': 'Cybersecurity', 'relation': 'Purpose in'}, {'source': 'Key', 'target': 'Sender and receiver', 'relation': 'Shared between'}, {'source': 'Symmetric encryption', 'target': 'Speed and efficiency', 'relation': 'Advantages of'}, {'source': 'Symmetric encryption', 'target': 'Secure key distribution', 'relation': 'Disadvantages of'}, {'source': 'Symmetric 

In [12]:
# step 3: prefetch key statement
key_candidates = []
for relation in graph_text:
    key_result = requests.post(
        url="http://18.206.229.27:8000/tree/create_key_statement",
        data=json.dumps({
            "source": relation["source"],
            "target": relation["target"],
            "label": relation["relation"],
            "cache": "true"
        }))
    key_text = json.loads(key_result.text)["data"]["key"]
    key_candidates.append(key_text)

In [14]:
# step 4: prefetch distractor statement
distractor_candidates = []
template = "According to the question, [restate the concepts being mentioned in the prompt] + [object] + is + [concept content] + Examples of + [object] + are + [example 1] + [example 2] + [example 3]."
for relation in graph_text:
    distractor_result = requests.post(
        url="http://18.206.229.27:8000/tree/create_distractor_statement",
        data=json.dumps({
            "source": relation["source"],
            "target": relation["target"],
            "label": relation["relation"],
            "template": template,
            "cache": "true"
        }))
    distractor_list = json.loads(distractor_result.text)["data"]["distractors"]
    for distractor in distractor_list:
        distractor_candidates.append(distractor)